In [10]:
from bs4 import BeautifulSoup as bs
import requests
import re
from pprint import pprint
from pymongo import MongoClient

ModuleNotFoundError: No module named 'pymongo'

In [ ]:
client = MongoClient('localhost', 27017)
db = client['dc06']
positions_collection = db.positions

In [ ]:
def find_position(position):
    return positions_collection.find(position)

In [ ]:
def insert_into_db(position):
    if !find_position:
        positions_collection.insertOne(position)

In [ ]:
def find_better_paid_positions(salary):
    positions_collection.find({ from_salary: { $gt: salary }, $or: [ up_to_salary: { $gt: salary } ] })

In [2]:
def to_number(str):
    filtered_string = ''.join(re.findall("\d+", str))
    return int(filtered_string)

In [3]:
def parse_position(position):
    scraped_position = { 'name': '', 'url': '', 'from_salary': None, 'up_to_salary': None, 'currency': '', 'from_site': url}
    
    uri = position.select('a[class*="f-test-link-"]')[0]['href']
    scraped_position['url'] = url + uri
    scraped_position['name'] = position.select('a[class*="f-test-link-"]')[0].text
    
    salary_string = position.find('span', { 'class': 'f-test-text-company-item-salary' }).contents[0].text.replace(u'\xa0', u' ')
    
    salary_array = salary_string.split(' ')
    scraped_position['currency'] = salary_array.pop(len(salary_array) - 1)
    
    if salary_array[0] == 'от':
        scraped_position['from_salary'] = to_number(salary_string)
    elif salary_array[0] == 'до':
        scraped_position['up_to_salary'] = to_number(salary_string)
    else:
        salary_range = salary_string.split('—')
        from_salary_str = ''.join(salary_range[0])
        scraped_position['from_salary'] = to_number(from_salary_str)
        to_salary_str = ''.join(salary_range[-1])
        scraped_position['up_to_salary'] = to_number(to_salary_str)
        
    scraped_positions.append(scraped_position)
    insert_into_db(scraped_position)

In [4]:
def parse_page_and_continue(response):
    soup = bs(response.text, 'html.parser')
    
    positions = soup.find_all('div', { 'class': 'f-test-vacancy-item' })

    for position in positions:
        parse_position(position)
        
    next_page = soup.find('a', href=True, text='Дальше')
    
    if next_page:
        pprint(next_page['href'])
        response = requests.get(url + next_page['href'])
        parse_page_and_continue(response)        



In [5]:
url = 'https://www.superjob.ru'
uri = '/vacancy/search/'
params = {
    'keywords': 'python',
    'payment_defined': 1
}

response = requests.get(url+uri, params=params)

In [6]:
scraped_positions = []

In [7]:
parse_page_and_continue(response)

'/vacancy/search/?keywords=python&payment_defined=1&page=2'
'/vacancy/search/?keywords=python&payment_defined=1&page=3'


In [8]:
len(scraped_positions)

59

In [9]:
scraped_positions

[{'name': 'Middle Python QA Engineer',
  'url': 'https://www.superjob.ru/vakansii/middle-python-qa-engineer-39416782.html',
  'from_salary': None,
  'up_to_salary': 150000,
  'currency': 'руб.',
  'from_site': 'https://www.superjob.ru'},
 {'name': 'Middle Data Engineer / Python разработчик',
  'url': 'https://www.superjob.ru/vakansii/middle-data-engineer-39413500.html',
  'from_salary': 104000,
  'up_to_salary': None,
  'currency': 'руб.',
  'from_site': 'https://www.superjob.ru'},
 {'name': 'Python developer',
  'url': 'https://www.superjob.ru/vakansii/python-developer-39307397.html',
  'from_salary': 120000,
  'up_to_salary': 140000,
  'currency': 'руб.',
  'from_site': 'https://www.superjob.ru'},
 {'name': 'Python developer',
  'url': 'https://www.superjob.ru/vakansii/python-developer-39082253.html',
  'from_salary': 180000,
  'up_to_salary': None,
  'currency': 'руб.',
  'from_site': 'https://www.superjob.ru'},
 {'name': 'Senior Python Backend Developer / WEB-разработчик',
  'url':

In [ ]:
find_better_paid_positions(170000)